In [1]:
print("devendra")

devendra


In [1]:
! {'pip install --quiet apache_beam'}

     |████████████████████████████████| 10.9 MB 9.5 MB/s 
     |████████████████████████████████| 508 kB 36.5 MB/s 
     |████████████████████████████████| 62 kB 1.4 MB/s 
     |████████████████████████████████| 2.4 MB 43.8 MB/s 
     |████████████████████████████████| 151 kB 14.4 MB/s 
     |████████████████████████████████| 47 kB 4.7 MB/s 
     |████████████████████████████████| 275 kB 66.1 MB/s 
     |████████████████████████████████| 1.0 MB 66.4 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.2+zzzcolab20220719082949 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.1 which is incompatible.
gym 0.17.3 requires cloudpickle<1.7.0,>=1.2.0, but you have cloudpickle 2.1.0 which is incompatible.


In [4]:
! {'mkdir -p data'}

In [7]:
! ls

data  sample_data


In [3]:
from google.colab import files
uploaded=files.upload()

Saving dept_data.txt to dept_data.txt


In [ ]:
! cat demo_code.txt

In [ ]:
! cat dept_data.txt

In [7]:
import apache_beam as beam

In [8]:
p1 = beam.Pipeline()

In [10]:
lines=(p1
       |beam.Create([1,2,3,4,5])
       |beam.io.WriteToText('data/Outcreate2')
)
p1.run()

In [ ]:
import apache_beam as beam
def SplitRow(element):
    return element.split(',')
def filtering(record):
  return record[3] == 'Accounts'
p2=beam.Pipeline()
Attendence_count=(p2
                 |'Read from File' >> beam.io.ReadFromText('dept_data.txt')
                 |'Split the records' >> beam.Map(SplitRow)
                 |'Get all rows having Account' >> beam.Filter(filtering)
                 |'Add 1 to each name ' >> beam.Map(lambda record: (record[1],1))
                 |'Count records' >> beam.CombinePerKey(sum)
                 |'Write Data in Output File'>>beam.io.WriteToText('data/output_new_final')
                 
)
p2.run()

# Sample the first 20 results, remember there are no ordering guarantees.
!{('head -n 20 data/output_new_final-00000-of-00001')}



In [ ]:
import apache_beam as beam
p=beam.Pipeline()
input_collection=(p
                 |beam.io.ReadFromText('dept_data.txt')
                 |"Split Rows" >>beam.Map(lambda record: record.split(','))
                 )

Account_Count=(input_collection
               |'Get all Accounts dept persons' >> beam.Filter(lambda record : record[3]=="Accounts")
               |'Pair each accounts employee with 1' >>  beam.Map(lambda record: ("Accounts, " +record[1], 1))
               |'Group and sum1' >> beam.CombinePerKey(sum)
               #| 'Write results for account' >> beam.io.WriteToText('data/Account')

)
HR_Count=(input_collection
               |'Get all HR dept persons' >> beam.Filter(lambda record : record[3]=="HR")
               |'Pair each HR employee with 1' >>  beam.Map(lambda record: ("HR, " +record[1], 1))
               |'Group and sum' >> beam.CombinePerKey(sum)
              # | 'Write results for account' >> beam.io.WriteToText('data/Account')

)
output =(
         (Account_Count,HR_Count)
    | beam.Flatten()
    | beam.io.WriteToText('data/both')
)
p.run()
  
# Sample the first 20 results, remember there are no ordering guarantees.
!{('head -n 20 data/both-00000-of-00001')}


#!{('head -n 20 data/HR-00000-of-00001')}

In [ ]:
# implementation of Pardo Transform
import apache_beam as beam
class SplitRow(beam.DoFn):
  def process(Self,element):
     return [element.split(',')]
class FilterAcc(beam.DoFn):
  def process(Self,element):
     if element[3] == 'Accounts':
      return [element] 
class Addone(beam.DoFn):
  def process(self, element):
    return [(element[3]+","+element[1], 1)]  
class Combine(beam.DoFn):
  def process(self, element):
    # return type -> list
    (key, values) = element           # [Marco, Accounts  [1,1,1,1....] , Rebekah, Accounts [1,1,1,1,....] ]
    return [(key, sum(values))]
p=beam.Pipeline()

Attendance_Count=(p
                  |beam.io.ReadFromText('dept_data.txt')
                  |beam.ParDo(SplitRow())
                  |beam.ParDo(FilterAcc())
                  |beam.ParDo(Addone())
                  |beam.ParDo(Combine())
                  |beam.io.WriteToText('data/output_new_final1')

)
p.run()
# Sample the first 20 results, remember there are no ordering guarantees.
#!{('head -n 20 data/output_new_final1-00000-of-00001')

In [3]:
import apache_beam as beam

with beam.Pipeline() as pipeline:
  plants = (
      pipeline
      | 'Gardening plants' >> beam.Create([
          'Strawberry Carrot Eggplant',
          '🍅Tomato 🥔Potato',
      ])
      | 'Split words' >> beam.FlatMap(str.split)
      | beam.Map(print))

Strawberry
Carrot
Eggplant
🍅Tomato
🥔Potato


In [1]:
from  google.cloud import bigquery